In [1]:
import numpy as np
topic={}
tfidf=dict(list(zip('cat dog apple lion NYC love'.split(),np.random.rand(6))))
tfidf

{'cat': 0.4484493901405543,
 'dog': 0.7004124484277804,
 'apple': 0.9642775099935134,
 'lion': 0.8590614411563071,
 'NYC': 0.4497812488383808,
 'love': 0.1279547049841938}

In [2]:
topic['petness']=(.3*tfidf['cat']+.3*tfidf['dog']+0*tfidf['apple']+0*tfidf['lion']-.2*tfidf['NYC']+.2*tfidf['love'])
topic['animalness']=(.1*tfidf['cat']+.1*tfidf['dog']-0.1*tfidf['apple']+0.5*tfidf['lion']-.1*tfidf['NYC']+.1*tfidf['love'])
topic['cityness']=(0*tfidf['cat']-.1*tfidf['dog']+0.2*tfidf['apple']-0.1*tfidf['lion']+.5*tfidf['NYC']+.1*tfidf['love'])

In [3]:
topic

{'petness': 0.28029324279966306,
 'animalness': 0.415806499050217,
 'cityness': 0.27459420795790374}

In [4]:
word_vector={}
word_vector['cat']=.3*topic['petness']+0.1*topic['animalness']+0.0*topic['cityness']
word_vector['dog']=.3*topic['petness']+0.1*topic['animalness']-0.1*topic['cityness']
word_vector['apple']=0*topic['petness']-0.1*topic['animalness']+0.2*topic['cityness']
word_vector['lion']=.0*topic['petness']+0.5*topic['animalness']-0.1*topic['cityness']
word_vector['NYC']=-0.2*topic['petness']+0.1*topic['animalness']+0.5*topic['cityness']
word_vector['love']=.2*topic['petness'] -0.1*topic['animalness']+0.1*topic['cityness']

In [5]:
word_vector

{'cat': 0.1256686227449206,
 'dog': 0.09820920194913023,
 'apple': 0.013338191686559052,
 'lion': 0.18044382872931813,
 'NYC': 0.12281910532404094,
 'love': 0.04193741945070129}

In [6]:
import pandas as pd
from nlpia.data.loaders import get_data
pd.options.display.width=120
sms=get_data('sms-spam')
print(sms)
index=['sms{}{}'.format(i,'!'*j) for (i,j) in zip(range(len(sms)),sms.spam)]
sms=pd.DataFrame(sms.values,columns=sms.columns,index=index)
sms['spam']=sms.spam.astype(int)
len(sms)

      spam                                               text
0        0  Go until jurong point, crazy.. Available only ...
1        0                      Ok lar... Joking wif u oni...
2        1  Free entry in 2 a wkly comp to win FA Cup fina...
3        0  U dun say so early hor... U c already then say...
4        0  Nah I don't think he goes to usf, he lives aro...
...    ...                                                ...
4832     1  This is the 2nd time we have tried 2 contact u...
4833     0               Will ü b going to esplanade fr home?
4834     0  Pity, * was in mood for that. So...any other s...
4835     0  The guy did some bitching but I acted like i'd...
4836     0                         Rofl. Its true to its name

[4837 rows x 2 columns]


4837

In [7]:
sms.spam.sum()


638

In [8]:
sms.head(10)

,spam,text
sms0,0,"Go until jurong point, crazy.. Available only ..."
sms1,0,Ok lar... Joking wif u oni...
sms2!,1,Free entry in 2 a wkly comp to win FA Cup fina...
sms3,0,U dun say so early hor... U c already then say...
sms4,0,"Nah I don't think he goes to usf, he lives aro..."
sms5!,1,FreeMsg Hey there darling it's been 3 week's n...
sms6,0,Even my brother is not like to speak with me. ...
sms7,0,As per your request 'Melle Melle (Oru Minnamin...
sms8!,1,WINNER!! As a valued network customer you have...
sms9!,1,Had your mobile 11 months or more? U R entitle...


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize.casual import casual_tokenize
tfidf_model=TfidfVectorizer(tokenizer=casual_tokenize)
tfidf_docs=tfidf_model.fit_transform(raw_documents=sms.text).toarray()

In [10]:
tfidf_docs.shape

(4837, 9232)

In [11]:
mask=sms.spam.astype(bool).values
spam_centriod=tfidf_docs[mask].mean(axis=0) #正常信息的tfidf的均值
print(spam_centriod)

[0.06377591 0.0041675  0.00056204 ... 0.         0.         0.        ]


In [12]:
mask

array([False, False,  True, ..., False, False, False])

In [13]:
ham_centroid=tfidf_docs[~mask].mean(axis=0) #垃圾短信的均值
ham_centroid

array([1.98493115e-02, 6.09435187e-03, 1.77747817e-04, ...,
       6.31869803e-05, 6.31869803e-05, 6.31869803e-05])

In [14]:
distance_vector=spam_centriod-ham_centroid
spamminess_score=tfidf_docs.dot(distance_vector)
print(distance_vector.round(2))
print(spam_centriod.round(2))
print(ham_centroid.round(2))

[ 0.04 -0.    0.   ... -0.   -0.   -0.  ]
[0.06 0.   0.   ... 0.   0.   0.  ]
[0.02 0.01 0.   ... 0.   0.   0.  ]


In [15]:
spamminess_score.round(2)

array([-0.01, -0.02,  0.04, ..., -0.01, -0.  ,  0.  ])

In [16]:
from sklearn.preprocessing import MinMaxScaler
sms['lda_score']=MinMaxScaler().fit_transform(spamminess_score.reshape(-1,1))

In [17]:
sms.head(6)

,spam,text,lda_score
sms0,0,"Go until jurong point, crazy.. Available only ...",0.227478
sms1,0,Ok lar... Joking wif u oni...,0.177888
sms2!,1,Free entry in 2 a wkly comp to win FA Cup fina...,0.718785
sms3,0,U dun say so early hor... U c already then say...,0.184565
sms4,0,"Nah I don't think he goes to usf, he lives aro...",0.286944
sms5!,1,FreeMsg Hey there darling it's been 3 week's n...,0.548003


In [18]:
sms['lda_predict']=(sms['lda_score']>.5).astype(int)

In [19]:
sum(sms['lda_predict']==1)

657

In [20]:
sms['spam lda_predict lda_score'.split()].round(2).head(6)

,spam,lda_predict,lda_score
sms0,0,0,0.23
sms1,0,0,0.18
sms2!,1,1,0.72
sms3,0,0,0.18
sms4,0,0,0.29
sms5!,1,1,0.55


In [21]:
(1.0-(sms.spam-sms.lda_predict).abs().sum()/len(sms)).round(3)

0.977

In [22]:
xs=sum(sms.spam==sms.lda_predict)/len(sms)
xs

0.9774653710977879

In [23]:
from pugnlp.stats import Confusion
Confusion(sms['spam lda_predict'.split()])

lda_predict,0,1
spam,,
0,4135,64
1,45,593


In [24]:
from nlpia.book.examples.ch04_catdog_lsa_3x6x16 import word_topic_vectors
word_topic_vectors.T.round(2)

100%|██████████| 263/263 [00:00<00:00, 854455.42it/s]


,cat,dog,apple,lion,nyc,love
top0,-0.58,-0.40,0.51,-0.27,0.44,-0.07
top1,-0.14,-0.31,-0.41,-0.11,0.09,0.84
top2,-0.32,0.76,-0.08,-0.54,0.04,0.11


In [25]:
from nlpia.book.examples.ch04_catdog_lsa_sorted import lsa_models,prettify_tdm
bow_svd,tfidf_svd=lsa_models()
prettify_tdm(**tfidf_svd)
tfidf_svd

100%|██████████| 263/263 [00:00<00:00, 820150.15it/s]


{'u':               0         1         2             3         4         5
 cat    0.036000 -0.748420 -0.529793  1.261930e-18  0.198828  0.344031
 dog    0.003867 -0.176567 -0.212674 -7.071068e-01 -0.489127 -0.429340
 apple  0.723958  0.259623 -0.393405 -2.220446e-16  0.376633 -0.334457
 lion   0.003867 -0.176567 -0.212674  7.071068e-01 -0.489127 -0.429340
 nyc    0.657583 -0.070319  0.266404  8.774703e-17 -0.508780  0.482506
 love   0.205291 -0.552419  0.634917 -4.317961e-17  0.285073 -0.410246,
 's': array([2.23578333, 1.62821839, 1.35725348, 0.83008327, 0.70772631,
        0.56402658]),
 'vt':                0             1             2             3             4             5             6             7   \
 d0   4.374410e-01  4.374410e-01  2.479758e-01  4.374410e-01  4.374410e-01  3.238049e-01  1.610175e-02  1.041454e-02   
 d1   8.894408e-02  8.894408e-02 -2.991345e-01  8.894408e-02  8.894408e-02  1.594522e-01 -4.596559e-01 -3.463379e-01   
 d2  -8.246610e-02 -8.246610e-02  4.

In [26]:
tdm=tfidf_svd['tdm']
tdm

,0,1,2,3,4,5,6,7,8,9,10
cat,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.0,0.557639,0.666353,0.000000,0.557639
dog,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.830083
apple,0.739990,0.739990,0.000000,0.739990,0.739990,1.0,0.0,0.000000,0.000000,0.000000,0.000000
lion,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.830083,0.000000,0.000000,0.000000
nyc,0.672618,0.672618,0.591367,0.672618,0.672618,0.0,0.0,0.000000,0.000000,0.591367,0.000000
love,0.000000,0.000000,0.806402,0.000000,0.000000,0.0,0.0,0.000000,0.745636,0.806402,0.000000


In [27]:
import numpy as np
U,S,Vt=np.linalg.svd(tdm)

In [28]:
import pandas as pd
pd.DataFrame(U,index=tdm.index).round(2)

,0,1,2,3,4,5
cat,0.04,-0.75,-0.53,0.00,0.20,0.34
dog,0.00,-0.18,-0.21,-0.71,-0.49,-0.43
apple,0.72,0.26,-0.39,-0.00,0.38,-0.33
lion,0.00,-0.18,-0.21,0.71,-0.49,-0.43
nyc,0.66,-0.07,0.27,0.00,-0.51,0.48
love,0.21,-0.55,0.63,-0.00,0.29,-0.41


In [29]:
print(S)

[2.23578333 1.62821839 1.35725348 0.83008327 0.70772631 0.56402658]


In [30]:
S.shape

(6,)

In [31]:
Vt.shape

(11, 11)

In [32]:
S.round(2)

array([2.24, 1.63, 1.36, 0.83, 0.71, 0.56])

In [33]:
s=np.zeros((len(U),len(Vt)))

In [34]:
pd.np.fill_diagonal(s,S)

In [35]:
s

array([[2.23578333, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 1.62821839, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.35725348, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.83008327, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.70772631,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.56402658, 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

In [36]:
pd.DataFrame(s).round(2)

,0,1,2,3,4,5,6,7,8,9,10
0,2.24,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0
1,0.00,1.63,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0
2,0.00,0.00,1.36,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0
3,0.00,0.00,0.00,0.83,0.00,0.00,0.0,0.0,0.0,0.0,0.0
4,0.00,0.00,0.00,0.00,0.71,0.00,0.0,0.0,0.0,0.0,0.0
5,0.00,0.00,0.00,0.00,0.00,0.56,0.0,0.0,0.0,0.0,0.0


In [37]:
a=[[1,0,0],[0,0,4],[2,4,0]]
a

[[1, 0, 0], [0, 0, 4], [2, 4, 0]]

In [38]:
q=[[0,0,1],[1,1,-8/15],[1,-1,-2/15]]
q

[[0, 0, 1], [1, 1, -0.5333333333333333], [1, -1, -0.13333333333333333]]

In [39]:
x=[[4,0,0],[0,-4,0],[0,0,1]]

In [40]:
import numpy as np
c=np.matmul(a,q)
c

array([[ 0.        ,  0.        ,  1.        ],
       [ 4.        , -4.        , -0.53333333],
       [ 4.        ,  4.        , -0.13333333]])

In [41]:
cx=np.matmul(q,x)
cx

array([[ 0.        ,  0.        ,  1.        ],
       [ 4.        , -4.        , -0.53333333],
       [ 4.        ,  4.        , -0.13333333]])

In [43]:
es=np.matmul(a,a.T)

AttributeError: 'list' object has no attribute 'T'

In [44]:
a=np.array(a)
a

array([[1, 0, 0],
       [0, 0, 4],
       [2, 4, 0]])

In [45]:
a.T

array([[1, 0, 2],
       [0, 0, 4],
       [0, 4, 0]])

In [46]:
np.matmul(a,a.T)

array([[ 1,  0,  2],
       [ 0, 16,  0],
       [ 2,  0, 20]])

In [47]:
np.matmul(a.T,a)

array([[ 5,  8,  0],
       [ 8, 16,  0],
       [ 0,  0, 16]])